In [2]:
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
import numpy as np


In [ ]:
from matplotlib import image
from matplotlib import pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, Conv2DTranspose, Concatenate, MaxPooling2D, Flatten, Dense


[Kaggle Dataset Link](https://www.kaggle.com/datasets/arka47/movie-frames-24k)

In [18]:
batch_size = 64
img_size = 240
subset_split = 3000

In [ ]:
#Converting image into LAB and extracting the dataset
import zipfile
zip_file_path = 'phto zip.zip'
extract_folder_path = 'D:\Image Classification\Image\photo extract'
os.makedirs(extract_folder_path, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_path)

print("Extraction completed.")

do = r"D:\Image Classification\Image\photo extract"
x = []
y = []
for image_file in os.listdir( do )[ 0 : subset_split ]:
    rgb_image = Image.open( os.path.join( do , image_file ) ).resize( ( img_size , img_size ) )
    rgb_img_array = (np.asarray( rgb_image ) ) / 255
    bw_image = rgb_image.convert( 'L' )
    bw_img_array = ( np.asarray( bw_image ).reshape( ( img_size , img_size , 1 ) ) ) / 255
    x.append( bw_img_array )
    y.append( rgb_img_array )

train_x, test_x, train_y, test_y = train_test_split( np.array(x) , np.array(y) , test_size=0.1 )

dataset = tf.data.Dataset.from_tensor_slices( ( train_x , train_y ) )
dataset = dataset.batch( batch_size )


In [ ]:
#building generator model
def build_sequential_generator_model(input_shape=(img_size, img_size, 1)):
    model = models.Sequential()
    
    # Encoder
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Bottleneck
    model.add(layers.Conv2D(512, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    
    # Decoder
    model.add(layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.Activation('relu'))
    
    # Output layer
    model.add(layers.Conv2D(3, (1, 1), padding='same'))
    model.add(layers.Activation('tanh'))
    
    return model

In [ ]:
#building discriminator model
def build_sequential_discriminator_model(input_shape=(256, 256, 3)):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(512, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten and dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [9]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()
mse = tf.keras.losses.MeanSquaredError()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output) - tf.random.uniform( shape=real_output.shape , maxval=0.1 ) , real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output) + tf.random.uniform( shape=fake_output.shape , maxval=0.1  ) , fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output , real_y):
    real_y = tf.cast( real_y , 'float32' )
    return mse( fake_output , real_y )

generator_optimizer = tf.keras.optimizers.Adam( 0.0005 )
discriminator_optimizer = tf.keras.optimizers.Adam( 0.0005 )



In [10]:
generator = build_sequential_generator_model()
discriminator = build_sequential_discriminator_model()

/home/d303222c-afb0-46cc-9db1-50eba8a3db60/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
@tf.function
def train_step( input_x , real_y ):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator( input_x , training=True)
        real_output = discriminator( real_y, training=True)
        generated_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss( generated_images , real_y )
        disc_loss = discriminator_loss( real_output, generated_output )

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [12]:
num_epochs = 700

for e in range( num_epochs ):
    print( e )
    for ( x , y ) in dataset:

        print( x.shape )
        train_step( x , y )

0


NameError: name 'dataset' is not defined

In [14]:
y = generator( test_x[0 : ] ).numpy()

NameError: name 'test_x' is not defined

In [29]:
for i in range(len(test_x)):
  plt.figure(figsize=(10,10))
  or_image = plt.subplot(3,3,1)
  or_image.set_title('bw Input', fontsize=16)
  plt.imshow( test_x[i].reshape((120,120)) , cmap='bw' )

  in_image = plt.subplot(3,3,2)
  image = Image.fromarray( ( y[i] * 255 ).astype( 'uint8' ) ).resize( ( 1024 , 1024 ) )
  image = np.asarray( image )
  in_image.set_title('Colorized Output', fontsize=16)
  plt.imshow( image )

  ou_image = plt.subplot(3,3,3)
  image = Image.fromarray( ( test_y[i] * 255 ).astype( 'uint8' ) ).resize( ( 1024 , 1024 ) )
  ou_image.set_title('Ground Truth', fontsize=16)
  plt.imshow( image )

  plt.show()

NameError: name 'test_x' is not defined